In [140]:
import warnings
warnings.filterwarnings('ignore')

In [141]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import scipy.stats as stats
import numpy as np
import geopandas as gpd
import seaborn as sns
from bokeh.palettes import Spectral6
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.io import output_notebook
from math import pi
from itertools import cycle



In [142]:
# Files to Load
EV_Data = Path("Electric_Vehicle_Population_Data.csv")

# Read sneaker data and store into Pandas DataFrames
EV_df = pd.read_csv(EV_Data)
EV_df.head()



,VIN,County,City,State,Postal_Code,Model_Year,Make,Model,Electric_Vehicle_Type,Clean_Alternative_Fuel_Vehicle_Eligibility,Electric_Range,Base_MSRP,Legislative_District,DOL_Vehicle_ID,Vehicle_Location,Electric_Utility,Census_Tract
0,5YJYGDEE1L,King,Seattle,WA,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,POINT (-122.30839 47.610365),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,7SAYGDEE9P,Snohomish,Bothell,WA,98021.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1.0,244285107,POINT (-122.179458 47.802589),PUGET SOUND ENERGY INC,5.306105e+10
2,5YJSA1E4XK,King,Seattle,WA,98109.0,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,270,0,36.0,156773144,POINT (-122.34848 47.632405),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,5YJSA1E27G,King,Issaquah,WA,98027.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,5.0,165103011,POINT (-122.03646 47.534065),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,5YJYGDEE5M,Kitsap,Suquamish,WA,98392.0,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,23.0,205138552,POINT (-122.55717 47.733415),PUGET SOUND ENERGY INC,5.303594e+10


In [143]:
EV_df.columns

Index(['VIN', 'County', 'City', 'State', 'Postal_Code', 'Model_Year', 'Make',
       'Model', 'Electric_Vehicle_Type',
       'Clean_Alternative_Fuel_Vehicle_Eligibility', 'Electric_Range',
       'Base_MSRP', 'Legislative_District', 'DOL_Vehicle_ID',
       'Vehicle_Location', 'Electric_Utility', 'Census_Tract'],
      dtype='object')

In [144]:
#Total number of records
EV_Counts = EV_df.value_counts("State")
EV_Counts

State
WA    177477
CA       101
VA        46
MD        31
TX        23
NC        16
IL        15
CO        12
AZ        10
FL        10
HI         9
NV         8
NJ         8
NY         7
SC         7
GA         6
AL         6
CT         6
OR         5
MO         5
MA         5
LA         4
PA         4
OH         4
UT         3
KY         3
BC         3
ID         3
DC         3
IN         3
RI         2
WY         2
NE         2
MN         2
MI         2
KS         2
AR         2
NM         1
NH         1
OK         1
AK         1
TN         1
IA         1
DE         1
AP         1
AE         1
Name: count, dtype: int64

In [145]:
EV_CarMake = EV_df.value_counts("Make")
fig = px.bar(EV_CarMake, x=EV_CarMake.index, y=EV_CarMake.values, labels={'x':'Make', 'y':'Count'}, title='Electric Vehicle Counts by Make')
fig.show()

In [146]:
#Registered EV by county
EV_Counties = EV_df.value_counts("County")
EV_Counties

County
King           92740
Snohomish      21001
Pierce         13782
Clark          10416
Thurston        6428
               ...  
Leavenworth        1
Las Animas         1
Larimer            1
Laramie            1
Yuba               1
Name: count, Length: 196, dtype: int64

In [147]:
EV_df = EV_df[EV_df['Model_Year'] <= 2023]
grouped_Electric_Vehicle_Type = EV_df.groupby(['Model_Year', 'Electric_Vehicle_Type']).size().reset_index(name='Count')
fig = px.line(grouped_Electric_Vehicle_Type, x='Model_Year', y='Count', color='Electric_Vehicle_Type', title='Trending of Vehicle Types over the Years')
fig.show()

In [148]:
# Filter
EV_df = EV_df[(EV_df['Model_Year'] >= 2019) & (EV_df['Model_Year'] <= 2023)]
data = EV_df['Model'].value_counts()
data = data.reset_index()
data.columns = ['Model', 'Count']

# proportions of each model
data['angle'] = data['Count']/data['Count'].sum() * 2*pi

# iterator
colors = cycle(Spectral6)
data['color'] = [next(colors) for _ in range(len(data))]

output_notebook()

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@Model: @Count", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Model', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None
show(p)

Loading BokehJS ...